In [2]:
import gender_guesser.detector as gender
import pandas as pd
import sqlite3 as sql

In [81]:
def convert(gen):
    male = ['male', 'mostly_male']
    female = ['female', 'mostly_female']
    na_ = ['unknown', 'andy']

    if gen in male:
        return 'm'
    elif gen in female:
        return 'f'
    elif gen in na_:
        return 'Na'

In [101]:
d = gender.Detector()
conn = sql.connect("author_gender.db")

df = pd.read_sql("""
select * from author_gender_dd ;
""", conn)

df = df.set_index("dhlabid")

aut = df['authors'].str.split("/", expand=True).stack()
gen = df['gender'].str.split("/", expand=True).stack()

joined = aut.to_frame("authors").join(gen.to_frame("gender"))
n = joined.authors.str.split(', ', expand=True)
fnames = n[1].str.split(" ", expand=True)[0]
joined['guessed_genders'] = fnames.apply(d.get_gender)
joined['gg_gender'] = joined.guessed_genders.apply(convert)
dfn = joined.drop('guessed_genders', axis=1).rename(columns = {'gender' : 'authority_reg_gender'})

In [105]:
df_unstacked = dfn.unstack()

In [156]:
dct = {}

In [157]:
for c in df_unstacked.columns.get_level_values(0).unique():
    split = df_unstacked[c].fillna('')
    dct[c] = split.agg('/'.join, axis=1).str.replace('//|/$', '', regex=True)

In [158]:
bigdf = pd.concat(dct, axis = 1)

,authors,authority_reg_gender,gg_gender
dhlabid,,,
100356077,"Olafsen , T.K.",Na,Na
100236738,"Buan , Edvard",Na,m
100082931,,Na,Na
100221491,"Knausgård , Karl Ove",m,m
100402613,"Egner , Thorbjørn",m,m
...,...,...,...
100354964,"Gjessing , Egil T.",Na,m
100061598,"Knutson , Tormod",Na,m
100480684,"Bygd , Sjur",m,m


In [142]:
df_unstacked.iloc[:, 2]

dhlabid
100356077                   NaN
100236738                   NaN
100082931                   NaN
100221491                   NaN
100402613                   NaN
                    ...        
100354964                   NaN
100061598                   NaN
100480684                   NaN
100087594                   NaN
100372566     Wirkola , Kirsten
Name: (authors, 2), Length: 570575, dtype: object

In [116]:
split = df_unstacked['authors'].fillna('')
authors = split.agg('/'.join, axis=1).str.replace('//|/$', '', regex=True)

In [117]:
split = df_unstacked['authority_reg_gender'].fillna('')
authority_reg_gender = split.agg('/'.join, axis=1).str.replace('//|/$', '', regex=True)

In [118]:
split = df_unstacked['gg_gender'].fillna('')
gg_gender = split.agg('/'.join, axis=1).str.replace('//|/$', '', regex=True)

In [126]:
bigdf = pd.concat([authors, authority_reg_gender, gg_gender], axis = 1)
bigdf.columns = ['authors', 'gender_aut_reg', 'gender_genderguesser']

In [132]:
bigdf.loc[bigdf['authors'] == ''].to_csv('lol.csv')

In [135]:
bigdf.loc[bigdf['authors'] != '']

,authors,gender_aut_reg,gender_genderguesser
dhlabid,,,
100356077,"Olafsen , T.K.",Na,Na
100236738,"Buan , Edvard",Na,m
100221491,"Knausgård , Karl Ove",m,m
100402613,"Egner , Thorbjørn",m,m
100217239,"Nygårdshaug , Gert",m,m
...,...,...,...
100354964,"Gjessing , Egil T.",Na,m
100061598,"Knutson , Tormod",Na,m
100480684,"Bygd , Sjur",m,m


In [133]:
bigdf.to_csv('author_gender_processed.csv')

In [122]:
dfn

authors authority_reg_gender gg_gender
dhlabid                                                                  
100356077 0                 Olafsen , T.K.                   Na        Na
100236738 0                  Buan , Edvard                   Na         m
100082931 0                                                  Na        Na
100221491 0           Knausgård , Karl Ove                    m         m
100402613 0              Egner , Thorbjørn                    m         m
...                                    ...                  ...       ...
100087594 0         Stensli , John Haakon                     f         m
          1                  Fossum , Knut                   Na         m
100372566 0                  Persen , Tor                     m         m
          1   Mikalsen , Anne Dagmar Biti                     f         f
          2              Wirkola , Kirsten                    f         f

[908159 rows x 3 columns]

In [110]:
gen

dhlabid
100356077                                 Olafsen , T.K./Na/Na
100236738                                   Buan , Edvard/Na/m
100082931                                               /Na/Na
100221491                             Knausgård , Karl Ove/m/m
100402613                                Egner , Thorbjørn/m/m
                                   ...                        
100354964                              Gjessing , Egil T./Na/m
100061598                                Knutson , Tormod/Na/m
100480684                                      Bygd , Sjur/m/m
100087594         Stensli , John Haakon / Fossum , Knutf/Nam/m
100372566    Persen , Tor / Mikalsen , Anne Dagmar Biti / W...
Length: 570575, dtype: object

In [ ]:
split1 = split.fillna('')
gen = split1.agg('/'.join, axis=1).str.replace('//|/$', '', regex=True).to_frame('gender')

In [75]:
df.iloc[:5].unstack()

authors               authority_reg_gender       \
                                  0             1                    0    1   
dhlabid                                                                       
100000000           Kinck , Hans E.           NaN                    m  NaN   
100000001             Holmsen , A.    Strøm , Ole                    f   Na   
100000002  Vinsnes , Johan Frederik           NaN                    m  NaN   
100000005        Petersen , Fredrik           NaN                   Na  NaN   

          gg_gender       
                  0    1  
dhlabid                   
100000000         m  NaN  
100000001        Na    m  
100000002         m  NaN  
100000005         m  NaN

In [ ]:
df_org

In [85]:
df2

authors authority_reg_gender gg_gender
dhlabid                                                             
100000000 0           Kinck , Hans E.                    m         m
100000001 0             Holmsen , A.                     f        Na
          1               Strøm , Ole                   Na         m
100000002 0  Vinsnes , Johan Frederik                    m         m
100000005 0        Petersen , Fredrik                   Na         m
...                               ...                  ...       ...
100579736 1            Dybwad , Jacob                   Na         m
100579738 0                 Tank , C.                    m        Na
100579739 0            Rieck , Gustav                   Na         m
100579740 0             Østbye , Tora                   Na         f
100579741 0          Amundsen , Roald                    m         m

[908182 rows x 3 columns]

In [88]:
df.loc[100082341]

,authors,authority_reg_gender,gg_gender
0,"Andreassen , Anders",Na,m
0,"Andreassen , Anders",Na,m
0,,Na,Na
0,,Na,Na


In [89]:
df[df.index.duplicated(keep=False)]

authors authority_reg_gender gg_gender
dhlabid                                                               
100082341 0         Andreassen , Anders                   Na         m
          0         Andreassen , Anders                   Na         m
          0                                               Na        Na
          0                                               Na        Na
100216201 0          Midelfart , Willi                    Na         m
          0          Midelfart , Willi                    Na         m
          0          Sørensen , Henrik                    Na         m
          0          Sørensen , Henrik                    Na         m
          1                Eidem , Odd                     m         m
          1                Eidem , Odd                    Na         m
          1   Butenschøn , Barthold A.                     m         m
          1   Butenschøn , Barthold A.                    Na         m
          2       Butenschøn , Ragnhild                    f         f
          2       Butenschøn , Ragnhild                    f         f
          2      Butenschøn , Ragnhild                     f         f
          2      Butenschøn , Ragnhild                     f         f
100224281 0            Schübeler , F.C.                   Na        Na
          0            Schübeler , F.C.                   Na        Na
          0            Schübeler , F.C.                   Na        Na
          0            Schübeler , F.C.                   Na        Na
100232626 0         Breidsvoll , Einar                     m         m
          0         Breidsvoll , Einar                    Na         m
          0                                                m        Na
          0                                               Na        Na
100494057 0         Steinsholt , Kjetil                    m         m
          0         Steinsholt , Kjetil                    m         m
          0         Steinsholt , Kjetil                    m         m
          0         Steinsholt , Kjetil                    m         m

In [73]:
df.authors.pivot(values='authors', columns=)

authors authority_reg_gender gg_gender
dhlabid                                                             
100000000 0           Kinck , Hans E.                    m         m
100000001 0             Holmsen , A.                     f        Na
          1               Strøm , Ole                   Na         m
100000002 0  Vinsnes , Johan Frederik                    m         m
100000005 0        Petersen , Fredrik                   Na         m
...                               ...                  ...       ...
100579736 1            Dybwad , Jacob                   Na         m
100579738 0                 Tank , C.                    m        Na
100579739 0            Rieck , Gustav                   Na         m
100579740 0             Østbye , Tora                   Na         f
100579741 0          Amundsen , Roald                    m         m

[908182 rows x 3 columns]

In [69]:
df.reindex()

authors authority_reg_gender gg_gender
dhlabid                                                             
100000000 0           Kinck , Hans E.                    m         m
100000001 0             Holmsen , A.                     f        Na
          1               Strøm , Ole                   Na         m
100000002 0  Vinsnes , Johan Frederik                    m         m
100000005 0        Petersen , Fredrik                   Na         m
...                               ...                  ...       ...
100579736 1            Dybwad , Jacob                   Na         m
100579738 0                 Tank , C.                    m        Na
100579739 0            Rieck , Gustav                   Na         m
100579740 0             Østbye , Tora                   Na         f
100579741 0          Amundsen , Roald                    m         m

[908182 rows x 3 columns]

In [65]:
df.authors

AttributeError: 'Series' object has no attribute 'stack'

In [67]:
df.stack().unstack()

ValueError: Index contains duplicate entries, cannot reshape

In [63]:
df.reset_index().loc[df.reset_index().duplicated(subset='dhlabid', keep=False)].reset_index()

,index,dhlabid,level_1,authors,authority_reg_gender,gg_gender
0,1,100000001,0,"Holmsen , A.",f,Na
1,2,100000001,1,"Strøm , Ole",Na,m
2,11,100000016,0,"Collett , Camilla",f,f
3,12,100000016,1,"Amundsen , Leiv",m,m
4,13,100000016,2,"Collett , P.J.",Na,Na
...,...,...,...,...,...,...
564061,908171,100579731,0,"Hansteen , Christopher",Na,m
564062,908172,100579731,1,"Lehmann , Jacob",Na,m
564063,908173,100579731,2,"Vega , Georg von",Na,m
564064,908176,100579736,0,"Bjørnson , Bjørnstjerne",f,Na


In [52]:
df.unstack(level=-1)

ValueError: Index contains duplicate entries, cannot reshape

In [49]:
df.reset_index().head()

,dhlabid,level_1,authors,authority_reg_gender,gg_gender
0,100000000,0,"Kinck , Hans E.",m,m
1,100000001,0,"Holmsen , A.",f,Na
2,100000001,1,"Strøm , Ole",Na,m
3,100000002,0,"Vinsnes , Johan Frederik",m,m
4,100000005,0,"Petersen , Fredrik",Na,m


In [51]:
df.reset_index().pivot(columns=['level_1'], values=['authors', 'authority_reg_gender', 'gg_gender'])

authors                                            \
level_1                        0                1    2    3    4    5    6    
0                 Kinck , Hans E.              NaN  NaN  NaN  NaN  NaN  NaN   
1                   Holmsen , A.               NaN  NaN  NaN  NaN  NaN  NaN   
2                             NaN      Strøm , Ole  NaN  NaN  NaN  NaN  NaN   
3        Vinsnes , Johan Frederik              NaN  NaN  NaN  NaN  NaN  NaN   
4              Petersen , Fredrik              NaN  NaN  NaN  NaN  NaN  NaN   
...                           ...              ...  ...  ...  ...  ...  ...   
908177                        NaN   Dybwad , Jacob  NaN  NaN  NaN  NaN  NaN   
908178                  Tank , C.              NaN  NaN  NaN  NaN  NaN  NaN   
908179             Rieck , Gustav              NaN  NaN  NaN  NaN  NaN  NaN   
908180              Østbye , Tora              NaN  NaN  NaN  NaN  NaN  NaN   
908181           Amundsen , Roald              NaN  NaN  NaN  NaN  NaN  NaN   

                        ... gg_gender                                          \
level_1   7    8    9   ...        49   50   51   52   53   54   55   56   57   
0        NaN  NaN  NaN  ...       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
1        NaN  NaN  NaN  ...       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
2        NaN  NaN  NaN  ...       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
3        NaN  NaN  NaN  ...       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
4        NaN  NaN  NaN  ...       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
...      ...  ...  ...  ...       ...  ...  ...  ...  ...  ...  ...  ...  ...   
908177   NaN  NaN  NaN  ...       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
908178   NaN  NaN  NaN  ...       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
908179   NaN  NaN  NaN  ...       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
908180   NaN  NaN  NaN  ...       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
908181   NaN  NaN  NaN  ...       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

              
level_1   58  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
4        NaN  
...      ...  
908177   NaN  
908178   NaN  
908179   NaN  
908180   NaN  
908181   NaN  

[908182 rows x 177 columns]

In [17]:
df = joined.drop('guessed_genders', axis=1).rename(columns = {'gender' : 'authority_reg_gender'})

In [ ]:
df.pivot(
    values = 
    index = 
    column = 
)

In [38]:
pd.IndexSlice

In [41]:
df.first_valid_index()

(100000000, 0)

In [ ]:
df.groupby(level=0)['authors'].mean()

In [ ]:
df.groupby(['year', 'month', 'item'])['value'].sum().unstack('item')

In [12]:
joined

authors gender guessed_genders gg_gender
dhlabid                                                               
100000000 0           Kinck , Hans E.      m            male         m
100000001 0             Holmsen , A.       f         unknown        Na
          1               Strøm , Ole     Na            male         m
100000002 0  Vinsnes , Johan Frederik      m            male         m
100000005 0        Petersen , Fredrik     Na            male         m
...                               ...    ...             ...       ...
100579736 1            Dybwad , Jacob     Na            male         m
100579738 0                 Tank , C.      m         unknown        Na
100579739 0            Rieck , Gustav     Na            male         m
100579740 0             Østbye , Tora     Na          female         f
100579741 0          Amundsen , Roald      m            male         m

[908182 rows x 4 columns]

In [6]:
df1 = pd.read_csv("/home/larsm/my_projects/metadata/Author_gender/author_gender_processed.csv", index_col=0)
df2 = pd.read_csv("/home/larsm/my_projects/metadata/Author_gender/author_gender_processed_test.csv", index_col=0)

In [11]:
df1.equals(df2)

False

In [12]:
df1

,authors,gender_aut_reg,gender_genderguesser
dhlabid,,,
100356077,"Olafsen , T.K.",Na,Na
100236738,"Buan , Edvard",Na,m
100221491,"Knausgård , Karl Ove",m,m
100402613,"Egner , Thorbjørn",m,m
100217239,"Nygårdshaug , Gert",m,m
...,...,...,...
100354964,"Gjessing , Egil T.",Na,m
100061598,"Knutson , Tormod",Na,m
100480684,"Bygd , Sjur",m,m


In [17]:
df2.loc[df2['authors'].notna() ]

,authors,gender_aut_reg,gender_genderguesser
dhlabid,,,
100356077,"Olafsen , T.K.",Na,Na
100236738,"Buan , Edvard",Na,m
100221491,"Knausgård , Karl Ove",m,m
100402613,"Egner , Thorbjørn",m,m
100217239,"Nygårdshaug , Gert",m,m
...,...,...,...
100354964,"Gjessing , Egil T.",Na,m
100061598,"Knutson , Tormod",Na,m
100480684,"Bygd , Sjur",m,m
